In [1]:
!pip install --upgrade  torchvision facenet-pytorch

INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 60.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 101.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 20.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 27.

In [2]:
import os
import shutil
import pandas as pd

# Paths
csv_path = "/kaggle/input/cacd-filtered-dataset/CACD_features_sex.csv"
images_root = "/kaggle/input/cacd-filtered-dataset/cacd_split/cacd_split"
output_root = "/kaggle/working/cacd_filtered_top200"
output_csv_path = "/kaggle/working/CACD_features_top200.csv"

# Create output directory
os.makedirs(output_root, exist_ok=True)

# Load CSV
df = pd.read_csv(csv_path)

# Extract person folder name from image filename (e.g., "AaronAshmore" from "25_Aaron_Ashmore_0005.jpg")
df["person"] = df["name"].apply(lambda x: "_".join(x.split("_")[1:-1]).replace("_", ""))

# Step 1: Compute age variation per person
age_variation = df.groupby("person")["age"].agg(lambda x: x.max() - x.min())

# Step 2: Select top 1000 persons with largest age variation
top_persons = age_variation.sort_values(ascending=False).head(200).index

# Step 3: Filter dataframe
filtered_df = df[df["person"].isin(top_persons)].copy()

# Step 4: Copy images into new folder structure
for _, row in filtered_df.iterrows():
    person = row["person"]
    img_name = row["name"]  # e.g., "25_Aaron_Ashmore_0005.jpg"
    
    src = os.path.join(images_root, person, img_name)
    dst_dir = os.path.join(output_root, person)
    dst = os.path.join(dst_dir, img_name)
    
    os.makedirs(dst_dir, exist_ok=True)
    
    if os.path.exists(src):
        shutil.copy(src, dst)

# Step 5: Save new CSV
filtered_df.to_csv(output_csv_path, index=False)

print(f"✅ Done! Filtered dataset saved at: {output_root}")
print(f"CSV saved at: {output_csv_path}")


✅ Done! Filtered dataset saved at: /kaggle/working/cacd_filtered_top200
CSV saved at: /kaggle/working/CACD_features_top200.csv


In [3]:
import os
from glob import glob
from PIL import Image
import torch
from facenet_pytorch import MTCNN
from tqdm import tqdm

# --- Configuration ---
# 1. Set the path to your root folder containing the identity subfolders.
SOURCE_DATA_DIR = "/kaggle/working/cacd_filtered_top200/"

# 2. Set the path where the processed (cropped and aligned) faces will be saved.
PROCESSED_DATA_DIR = "/kaggle/working/processed_aifr_faces/"

# 3. MTCNN parameters
# The size of the output image. ArcFace models are often trained on 112x112 or 160x160.
# Let's use 160x160, which is standard for FaceNet-based models.
IMAGE_SIZE = 160
# Margin adds padding around the detected face bounding box.
MARGIN = 20

# --- GPU/CPU Device Setup ---
# Automatically select a GPU if available, otherwise fall back to CPU.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Running on device: {device}')

# --- Instantiate MTCNN ---
# keep_all=False ensures that we only get the most confident face detection if multiple faces are in the image.
# post_process=True normalizes the output tensor image to the range [-1, 1], which is standard for many models.
mtcnn = MTCNN(
    image_size=IMAGE_SIZE,
    margin=MARGIN,
    keep_all=False,
    post_process=True,
    device=device
)

# --- Main Processing Loop ---
def process_and_save_faces():
    """
    Finds all images in the source directory, detects faces, aligns them,
    and saves them to the processed directory, maintaining the folder structure.
    """
    print("Starting face detection and alignment process...")

    # Use os.walk to recursively go through the directory structure
    for dirpath, _, filenames in os.walk(SOURCE_DATA_DIR):
        # Filter for image files
        image_paths = [os.path.join(dirpath, f) for f in filenames if f.endswith(('.jpg', '.jpeg', '.png'))]

        if not image_paths:
            continue

        # Create the corresponding subfolder in the destination directory
        relative_path = os.path.relpath(dirpath, SOURCE_DATA_DIR)
        dest_folder = os.path.join(PROCESSED_DATA_DIR, relative_path)
        os.makedirs(dest_folder, exist_ok=True)

        print(f"Processing folder: {relative_path}")

        # Use tqdm for a progress bar
        for image_path in tqdm(image_paths, desc=f"  -> {relative_path}"):
            try:
                # Open the image
                img = Image.open(image_path).convert('RGB')

                # Use MTCNN to detect and align the face
                face_tensor = mtcnn(img)

                # Check if a face was detected
                if face_tensor is not None:
                    # Construct the output path
                    filename = os.path.basename(image_path)
                    output_path = os.path.join(dest_folder, filename)

                    # Convert tensor to a savable image format
                    # The tensor is [-1, 1], so we need to scale it to [0, 255]
                    face_tensor = (face_tensor + 1) / 2.0
                    # Use torchvision's utility to save the tensor as an image
                    from torchvision.utils import save_image
                    save_image(face_tensor, output_path)

                else:
                    # If no face is detected, print a warning
                    print(f"\n[WARNING] No face detected in: {image_path}. Skipping.")

            except Exception as e:
                # Catch potential errors like corrupted image files
                print(f"\n[ERROR] Could not process {image_path}. Reason: {e}. Skipping.")

    print("\nProcessing complete!")
    print(f"All aligned faces are saved in: {PROCESSED_DATA_DIR}")



process_and_save_faces()

Running on device: cuda:0
Starting face detection and alignment process...
Processing folder: AmberTamblyn


  -> AmberTamblyn: 100%|██████████| 100/100 [00:03<00:00, 26.92it/s]


Processing folder: AbbieCornish


  -> AbbieCornish: 100%|██████████| 108/108 [00:03<00:00, 34.37it/s]


Processing folder: AllySheedy


...

Processing folder: AnnaPopplewell


  -> AnnaPopplewell: 100%|██████████| 102/102 [00:02<00:00, 34.81it/s]


Processing folder: BillSkarsgård


  -> BillSkarsgård: 100%|██████████| 67/67 [00:01<00:00, 35.28it/s]


Processing complete!
All aligned faces are saved in: /kaggle/working/processed_aifr_faces/


In [5]:
# --- 1.2. Import libraries ---
import os
import random
import shutil
from collections import defaultdict
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1, MTCNN
from tqdm.notebook import tqdm  # Use notebook-friendly tqdm
import matplotlib.pyplot as plt
import seaborn as sns


# --- 1.3. Global Configuration ---

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
# Ensure deterministic behavior for cuDNN
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on device: {device}")

# --- Configuration Parameters ---
# FIXME: UPDATE THIS PATH to your folder of processed faces from Phase 1
PROCESSED_DATA_DIR = "/kaggle/working/processed_aifr_faces/"

# Model parameters
EMBEDDING_DIM = 512 # Output dimension of the InceptionResnetV1
NUM_EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# --- Sanity Check ---
# Check if the processed data directory exists
if not os.path.isdir(PROCESSED_DATA_DIR):
    print("="*50)
    print(f"ERROR: The directory '{PROCESSED_DATA_DIR}' was not found.")
    print("Please make sure you have run Phase 1 (pre-processing) and have set the correct path.")
    print("="*50)
else:
    print(f"Successfully found processed data directory: {PROCESSED_DATA_DIR}")

Running on device: cuda
Successfully found processed data directory: /kaggle/working/processed_aifr_faces/


In [6]:
def prepare_data_splits(base_dir, test_size=0.1, val_size=0.1):
    """
    Splits identities into train, validation, and test sets.
    Ensures that no identity appears in more than one set.
    """
    print("Preparing data splits...")
    all_identity_folders = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

    # Split identities
    train_identities, test_identities = train_test_split(all_identity_folders, test_size=test_size, random_state=SEED)
    relative_val_size = val_size / (1.0 - test_size)
    train_identities, val_identities = train_test_split(train_identities, test_size=relative_val_size, random_state=SEED)

    print(f"Total identities: {len(all_identity_folders)}")
    print(f"Training identities: {len(train_identities)}")
    print(f"Validation identities: {len(val_identities)}")
    print(f"Test identities: {len(test_identities)}")

    # Create mapping from identity name to integer label for training
    identity_to_label = {identity: i for i, identity in enumerate(train_identities)}

    def get_filepaths_and_labels(identity_list, is_training=False):
        filepaths, labels = [], []
        for identity in identity_list:
            identity_path = os.path.join(base_dir, identity)
            images = os.listdir(identity_path)
            for img in images:
                filepaths.append(os.path.join(identity_path, img))
                labels.append(identity_to_label[identity] if is_training else identity)
        return filepaths, labels

    train_files, train_labels = get_filepaths_and_labels(train_identities, is_training=True)
    val_files, val_labels = get_filepaths_and_labels(val_identities)
    test_files, test_labels = get_filepaths_and_labels(test_identities)

    num_train_classes = len(train_identities)
    return (train_files, train_labels), (val_files, val_labels), (test_files, test_labels), num_train_classes

# --- Execute the split ---
# This will be run once and the results will be stored in variables for later cells.
(train_files, train_labels), (val_files, val_labels), (test_files, test_labels), NUM_TRAIN_CLASSES = prepare_data_splits(PROCESSED_DATA_DIR)

Preparing data splits...
Total identities: 200
Training identities: 160
Validation identities: 20
Test identities: 20


In [7]:
class FaceDataset(Dataset):
    def __init__(self, filepaths, labels, transform=None):
        self.filepaths, self.labels, self.transform = filepaths, labels, transform

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, idx):
        img_path = self.filepaths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

def generate_evaluation_pairs(files, labels, num_pairs=5000):
    pairs, is_same_list = [], []
    identity_dict = defaultdict(list)
    for file, label in zip(files, labels):
        identity_dict[label].append(file)
    identities = list(identity_dict.keys())

    for _ in tqdm(range(num_pairs), desc="Generating pairs"):
        choice = random.choice([True, False])
        if choice and len(identities) > 0: # Positive pair
            identity = random.choice([id for id, files in identity_dict.items() if len(files) >= 2])
            img1_path, img2_path = random.sample(identity_dict[identity], 2)
            is_same_list.append(1)
        else: # Negative pair
            id1, id2 = random.sample(identities, 2)
            img1_path = random.choice(identity_dict[id1])
            img2_path = random.choice(identity_dict[id2])
            is_same_list.append(0)
        pairs.append((img1_path, img2_path))

    return pairs, is_same_list

# --- Define Transforms ---
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
])
eval_transform = transforms.Compose([transforms.ToTensor()])

# --- Create Datasets and Dataloaders ---
train_dataset = FaceDataset(train_files, train_labels, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)

# Generate validation pairs (this might take a moment)
val_pairs, val_is_same = generate_evaluation_pairs(val_files, val_labels)

print("Datasets, transforms, and dataloaders are ready.")

Generating pairs:   0%|          | 0/5000 [00:00<?, ?it/s]

Datasets, transforms, and dataloaders are ready.


In [8]:
class ArcFaceLoss(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50):
        super(ArcFaceLoss, self).__init__()
        self.in_features, self.out_features, self.s, self.m = in_features, out_features, s, m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)
        self.cos_m, self.sin_m = np.cos(m), np.sin(m)
        self.th = np.cos(np.pi - m)
        self.mm = np.sin(np.pi - m) * m

    def forward(self, embedding, label):
        cosine = F.linear(F.normalize(embedding), F.normalize(self.weight))
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        phi = cosine * self.cos_m - torch.sqrt((1.0 - cosine.pow(2)).clamp(0, 1)) * self.sin_m
        phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        return output * self.s

class FaceRecognitionModel(nn.Module):
    def __init__(self, num_classes, embedding_dim=EMBEDDING_DIM, pretrained='vggface2'):
        super(FaceRecognitionModel, self).__init__()
        self.backbone = InceptionResnetV1(pretrained=pretrained)
        self.arc_face = ArcFaceLoss(in_features=embedding_dim, out_features=num_classes)

    def forward(self, image, label):
        embedding = self.backbone(image)
        return self.arc_face(embedding, label)

    def get_embedding(self, image):
        return self.backbone(image)



In [9]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    progress_bar = tqdm(dataloader, desc="Training", leave=True)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(images, labels)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    return total_loss / len(dataloader)

def validate_one_epoch(model, val_pairs, val_is_same, transform, device):
    model.eval()
    similarities = []
    with torch.no_grad():
        for img1_path, img2_path in tqdm(val_pairs, desc="Validating", leave=True):
            img1 = transform(Image.open(img1_path).convert('RGB')).unsqueeze(0).to(device)
            img2 = transform(Image.open(img2_path).convert('RGB')).unsqueeze(0).to(device)
            emb1, emb2 = model.get_embedding(img1), model.get_embedding(img2)
            similarities.append(F.cosine_similarity(emb1, emb2).item())

    fpr, tpr, thresholds = roc_curve(val_is_same, similarities)
    return auc(fpr, tpr)

print("Training and validation functions defined.")

Training and validation functions defined.


In [10]:
# --- Initialize Model, Loss, and Optimizer ---
model = FaceRecognitionModel(num_classes=NUM_TRAIN_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


  0%|          | 0.00/107M [00:00<?, ?B/s]

In [11]:
model.eval()

FaceRecognitionModel(
  (backbone): InceptionResnetV1(
    (conv2d_1a): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2a): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2b): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2d_3b): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, aff

In [12]:

# --- Training Loop ---
best_val_auc = 0.0
print("\n Starting Training...")
for epoch in range(NUM_EPOCHS):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_auc = validate_one_epoch(model, val_pairs, val_is_same, eval_transform, device)
    scheduler.step()

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {train_loss:.4f} | Val AUC: {val_auc:.4f} | LR: {scheduler.get_last_lr()[0]:.1e}")

    if val_auc > best_val_auc:
        best_val_auc = val_auc
        print(f"New best validation AUC! Saving model to 'best_aifr_model.pth'")
        torch.save(model.state_dict(), 'best_aifr_model.pth')

print(" Training finished.")


 Starting Training...


Training:   0%|          | 0/390 [00:00<?, ?it/s]

Validating:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 1/20 | Train Loss: 17.4158 | Val AUC: 0.8807 | LR: 1.0e-03
New best validation AUC! Saving model to 'best_aifr_model.pth'


Training:   0%|          | 0/390 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 19/20 | Train Loss: 1.3668 | Val AUC: 0.8899 | LR: 1.0e-06


Training:   0%|          | 0/390 [00:00<?, ?it/s]

Validating:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 20/20 | Train Loss: 1.3320 | Val AUC: 0.8890 | LR: 1.0e-07
 Training finished.
